# Imports

In [1]:
from utils import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import numpy as np
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

C:\Users\Joely\anaconda3\envs\comp_genomics\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Joely\anaconda3\envs\comp_genomics\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


# Data Loading

In [2]:
seqs, labels = load_data_train()

# Model

### Experimentation Setup

In [16]:
curr_cutoff = None
curr_kmers = None

In [38]:
# experiments
# NOTE THAT in_channels_conv1 is always the seq_cutoff length
hyperparams = [
    # EXPERIMENT 0
    {'in_channels_conv1': 1000,     'out_channels_conv1': 300,
     'in_channels_conv2': 150,     'out_channels_conv2': 300,
     'in_channels_conv3': 150,     'out_channels_conv3': 300,
     'in_channels_fc1': 150,     'in_channels_fc2': 300, 
     'cutoff': 1000,     'kmer_size': None,
     'kernel_size': 20,     'stride': 20,
     'decay': 0.01,
     'epochs': 10,
     'train_acc': 0.33,     'test_acc': 0.33,     'time': None
    },
    
    # EXPERIMENT 1
    {'in_channels_conv1': 2000,     'out_channels_conv1': 600,
     'in_channels_conv2': 300,     'out_channels_conv2': 600,
     'in_channels_conv3': 300,     'out_channels_conv3': 600,
     'in_channels_fc1': 300,     'in_channels_fc2': 600, 
     'cutoff': 2000,     'kmer_size': None,
     'kernel_size': 20,     'stride': 20,
     'decay': 0.01,
     'epochs': 10,
     'train_acc': 0.36,     'test_acc': 0.34,     'time': "5hr"
    },  
    
    # EXPERIMENT 2
    {'in_channels_conv1': 1000,     'out_channels_conv1': 300,
     'in_channels_conv2': 150,     'out_channels_conv2': 75,
     'in_channels_conv3': 75//2,     'out_channels_conv3': 75//2,
     'in_channels_fc1': 38,     'in_channels_fc2': 300, 
     'cutoff': 1000,     'kmer_size': None,
     'kernel_size': 20,     'stride': 20,
     'decay': 0.01,
     'epochs': 10,
     'train_acc': 0.27,     'test_acc': 0.27,     'time': "1h 1m 40s"
    },
    
    # EXPERIMENT 3
    {'in_channels_conv1': 998,     'out_channels_conv1': 998//2,
     'in_channels_conv2': 250,     'out_channels_conv2': 998//2,
     'in_channels_conv3': 250,     'out_channels_conv3': 998//2,
     'in_channels_fc1': 250,     'in_channels_fc2': 998//2, 
     'cutoff': 1000,     'kmer_size': 3,
     'kernel_size': 20,     'stride': 20,
     'decay': 0.01,
     'epochs': 10,
     'train_acc': 0.36,     'test_acc': 0.35,     'time': '5h 11m 16s'
    },
    
    # EXPERIMENT 4
    {'in_channels_conv1': 1498,     'out_channels_conv1': 1498//2,
     'in_channels_conv2': 375,     'out_channels_conv2': 1498//2,
     'in_channels_conv3': 375,     'out_channels_conv3': 1498//2,
     'in_channels_fc1': 375,     'in_channels_fc2': 1498//2, 
     'cutoff': 1500,     'kmer_size': 3,
     'kernel_size': 20,     'stride': 20,
     'decay': 0.01,
     'epochs': 10,
     'train_acc': None,     'test_acc': None,     'time': None
    },
    
]

i = 4
hp = hyperparams[i]

In [39]:
if curr_cutoff is None or curr_cutoff != hp['cutoff'] or curr_kmers != hp['kmer_size']:
        x = prep_data(seqs, labels, cutoff_len=hp['cutoff'], kmer_size=hp['kmer_size'])
        train_dataloader, test_dataloader, new_seqs, voc2ind, tokenized_seqs, labels = x
        FEATURE_SIZE = tokenized_seqs.shape[1]
else:
    print('already prepared for this in_channels_conv1')
    
curr_cutoff = hp['in_channels_conv1']

data prepared
Finding Max Sequence Length
Max sequence length found: 1498
Min sequence length found: 18
Begin Padding
Change to a numpy array
tokenized sequenced
Begin prepare labels
tokenized labels


 [[ 1.  2.  3. ... 40. 50. 46.]
 [45. 44. 29. ...  0.  0.  0.]
 [65. 66. 33. ...  0.  0.  0.]
 ...
 [33. 20. 22. ... 41. 41. 12.]
 [58.  7. 25. ... 26. 33. 49.]
 [29. 42. 35. ... 40. 50. 46.]] 

 {'<pad>': 0, ('C', 'A', 'T'): 1, ('A', 'T', 'G'): 2, ('T', 'G', 'C'): 3, ('G', 'C', 'A'): 4, ('A', 'T', 'T'): 5, ('T', 'T', 'A'): 6, ('T', 'A', 'G'): 7, ('A', 'G', 'T'): 8, ('G', 'T', 'T'): 9, ('T', 'A', 'T'): 10, ('T', 'A', 'A'): 11, ('A', 'A', 'T'): 12, ('A', 'T', 'A'): 13, ('G', 'T', 'A'): 14, ('A', 'T', 'C'): 15, ('T', 'C', 'A'): 16, ('C', 'A', 'A'): 17, ('T', 'A', 'C'): 18, ('A', 'C', 'G'): 19, ('C', 'G', 'G'): 20, ('G', 'G', 'G'): 21, ('G', 'G', 'T'): 22, ('G', 'T', 'C'): 23, ('T', 'T', 'C'): 24, ('A', 'G', 'C'): 25, ('G', 'C', 'C'): 26, ('C', 'C', 'C'): 27, ('C', 'C', 'A'): 28, ('T', 'G',

In [ ]:
class DSC_by_CNN(nn.Module):
    def __init__(self, vocab_size, feature_size, num_classes):
        


        super(DSC_by_CNN, self).__init__()
        # Number of vocab words
        self.vocab_size = vocab_size
        # embeded_dim: Dimension of word vectors.
        self.feature_size = feature_size

        self.encoder = nn.Embedding(self.vocab_size, self.feature_size)
        

        self.conv1 = nn.Conv1d(in_channels=hp['in_channels_conv1'],
                               out_channels=hp['out_channels_conv1'], 
                               kernel_size=hp['kernel_size'],
                               stride = hp['stride'],
                               padding = 1)
        
        self.conv2 = nn.Conv1d(in_channels=hp['in_channels_conv2'],
                               out_channels=hp['out_channels_conv2'], 
                               kernel_size=hp['kernel_size'],
                               stride = hp['stride'],
                               padding = 1)
        
        self.conv3 = nn.Conv1d(in_channels=hp['in_channels_conv3'],
                               out_channels=hp['out_channels_conv3'], 
                               kernel_size=hp['kernel_size'],
                               stride = hp['stride'],
                               padding = 1)

        # Activation
        self.act = nn.ReLU()

        # Pooling
        self.pool = nn.MaxPool2d(3, 2, 1)

        # fully connected layer
        self.fc1 = nn.Linear(hp['in_channels_fc1'], hp['in_channels_fc2'])
        self.fc2 = nn.Linear(hp['in_channels_fc2'], num_classes)

        # dropout
        self.drop = nn.Dropout(p=0.5)


    # seen in the network pytorch tutorial
    # I've been using this function for years, no idea what it does.
    # Would it kill the pytorch tutorial people to document this guy?
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features        


    def forward(self, x):
        # print("forward")
        # The following line is required to send it through the encoder. 
        # because indexes are neccesary for use in the encoder
        # x = torch.tensor(x).to(torch.int64)
        x = x.clone().detach().requires_grad_(True).to(torch.int64)
        
    
        # Encode. 
        # Output shape: (b, max_len, embedded_dim)
        x = self.encoder(x)
  
        # Permute shuffles dimensions. Do this to satisfy requirments of nn.Conv1d
        # Output shape: (b, embedded_dim, max_len)
        x = x.permute(0, 2, 1)

        # convolutional layer 1
        x = self.conv1(x)
        x = self.act(x)                         
        x = self.pool(x)

        # convolutional layer 2
        x = self.conv2(x)
        x = self.act(x)
        x = self.pool(x)
        
        # convolutional layer 3

        # flattening
        x = x.view(-1, self.num_flat_features(x))

        # Tring to figure out the shape
        #print(x.shape)

        # fully linear layer + dropout
        x = self.drop(x)
        # x = self.softm(x)
        x = self.fc1(x)
        
        # fully linear layer
        x = self.fc2(x)

        return x
    
start_time = time.time()
# Creating net and training
net = DSC_by_CNN(len(voc2ind), FEATURE_SIZE, len(set(labels)))
losses = train(net, train_dataloader, device, epochs=hp['epochs'], lr=0.01, decay = hp['decay'])


# Evaluation
plot_losses(losses, title = "Convolutional Network Loss")
print_eval(net, train_dataloader, test_dataloader)
print()

# time to complete
end_time = time.time()
print('Time to complete', time_str(end_time - start_time))

[1,   100] loss: 5.671
[1,   200] loss: 5.269
[1,   300] loss: 4.897
[1,   400] loss: 4.766
[1,   500] loss: 4.606
[1,   600] loss: 4.500
[1,   700] loss: 4.332
[1,   800] loss: 4.282
[1,   900] loss: 4.195
[1,  1000] loss: 4.167
[1,  1100] loss: 4.095
[2,   100] loss: 3.839
[2,   200] loss: 3.869
[2,   300] loss: 3.849
[2,   400] loss: 3.762
[2,   500] loss: 3.882
[2,   600] loss: 3.789
[2,   700] loss: 3.827
[2,   800] loss: 3.875
[2,   900] loss: 3.733
[2,  1000] loss: 3.823
[2,  1100] loss: 3.842
[3,   100] loss: 3.653
[3,   200] loss: 3.625
[3,   300] loss: 3.645
[3,   400] loss: 3.688
[3,   500] loss: 3.767
[3,   600] loss: 3.756
[3,   700] loss: 3.784
[3,   800] loss: 3.765
[3,   900] loss: 3.841
[3,  1000] loss: 3.807
[3,  1100] loss: 3.801
[4,   100] loss: 3.679
[4,   200] loss: 3.781
[4,   300] loss: 3.736
[4,   400] loss: 3.788
[4,   500] loss: 3.836
[4,   600] loss: 3.769
